# Download from GitHUB 
# Source data format check 

In [1]:
#DOWNLOAD The files from git 
#!git clone https://github.com/momenton/momenton-code-test-movietweetings.git 
!pip install pandasql

In [2]:
#!mv /root/golive/snapshots/100k/*.dat .
!ls -l *.dat 
!head *.dat

-rw-r--r--. 1 root root  542192 Jul  3 19:48 movies.dat
-rw-r--r--. 1 root root 2945101 Jul  3 19:48 ratings.dat
-rw-r--r--. 1 root root  262574 Jul  3 19:48 users.dat
==> movies.dat <==
0002844::Fantômas - À l'ombre de la guillotine (1913)::Crime|Drama
0004936::The Bank (1915)::Comedy|Short
0004972::The Birth of a Nation (1915)::Drama|History|Romance|War
0005078::The Cheat (1915)::Drama
0006684::The Fireman (1916)::Short|Comedy
0006689::The Floorwalker (1916)::Short|Comedy
0007264::The Rink (1916)::Comedy|Short
0007880::Easy Street (1917)::Short|Comedy
0008133::The Immigrant (1917)::Short|Comedy|Drama|Romance
0009018::A Dog's Life (1918)::Short|Comedy

==> ratings.dat <==
1::1074638::7::1365029107
1::1853728::8::1366576639
2::0104257::8::1364690142
2::1259521::8::1364118447
2::1991245::7::1364117717
3::1300854::7::1368156300
4::1457767::2::1377910113
5::0903624::9::1377745112
5::1213663::9::1377679509
6::1291584::6::1370880651

==> users.dat <==
1::18405182
2::31260677
3::138805259
4:

# Import Libraries , Load *.dat and cleanup , load to  Data frames 

In [3]:
import pandas as pd 
import numpy as np 
import re 
from datetime import datetime
import pandasql as ps

import seaborn as sns

import matplotlib
sns.set(style="ticks", color_codes=True)

In [4]:
#LOAD Movies.dat and Clean up 
header = ["ID", "NAME_YEAR", "GENRE_LIST"]
movies_df=pd.read_csv('movies.dat',sep='::',engine='python',names=header)
movies_df.head(5)

,ID,NAME_YEAR,GENRE_LIST
0,2844,Fantômas - À l'ombre de la guillotine (1913),Crime|Drama
1,4936,The Bank (1915),Comedy|Short
2,4972,The Birth of a Nation (1915),Drama|History|Romance|War
3,5078,The Cheat (1915),Drama
4,6684,The Fireman (1916),Short|Comedy


In [5]:
temp_df_1 = movies_df["NAME_YEAR"].str.split("(", n = 1, expand = True) 
temp_df_2= temp_df_1[1].str.split(")",n = 1 , expand = True) 
movies_df["NAME"] = temp_df_1 [0]
movies_df["YEAR"] = temp_df_2 [0]
movies_df.drop(columns =["NAME_YEAR"], inplace = True)
print(movies_df.count())
movies_df.dropna( axis=0, how='any', subset=['GENRE_LIST'], inplace=True)
print(movies_df.count())
movies_df.head(5)
#movies_df['GENRE_LIST'].isnull().values.any()

ID            10506
GENRE_LIST    10440
NAME          10506
YEAR          10506
dtype: int64
ID            10440
GENRE_LIST    10440
NAME          10440
YEAR          10440
dtype: int64


,ID,GENRE_LIST,NAME,YEAR
0,2844,Crime|Drama,Fantômas - À l'ombre de la guillotine,1913
1,4936,Comedy|Short,The Bank,1915
2,4972,Drama|History|Romance|War,The Birth of a Nation,1915
3,5078,Drama,The Cheat,1915
4,6684,Short|Comedy,The Fireman,1916


In [6]:
# Get unique list of All Genres
temp_df_3=movies_df["GENRE_LIST"].str.split("|", n = -1, expand = True)
arr = []
unique_list = []    
for i in range(len(temp_df_3.columns) - 1 ):
    x= list (set(temp_df_3[i].dropna()))
    for val in x:
           if val not in unique_list: 
                unique_list.append(val)

df_unique_genre = pd.DataFrame(unique_list)
df_unique_genre.columns=['genre']
df_unique_genre

,genre
0,Mystery
1,Thriller
2,Documentary
3,Fantasy
4,Short
5,Film-Noir
6,Adventure
7,Adult
8,Family
9,Romance


In [7]:
# DUplicate the rows based on number of Genre (e.g if a move is in 3 Genre , 
#3 records of the same movie will be inserted into the new datafraem )


movies_df_new =pd.DataFrame()
data = []
for index, row in movies_df.iterrows():
    genre_str =(row["GENRE_LIST"])
    #print(len(genre_str.split('|')))
    for genre in (genre_str.split('|')):
        #print (row["ID"], row["NAME"], row["YEAR"],genre)
        v_dict = {'ID':row["ID"] ,'GENRE':genre, 'NAME':row["NAME"],'YEAR':row["YEAR"]}
        #print(v_dict)
        data.append(v_dict)
        #print(movies_df_new)
movies_df_new = movies_df_new.append(data,ignore_index=True)     
movies_df_new.head(5)

        
   
    


,ID,GENRE,NAME,YEAR
0,2844,Crime,Fantômas - À l'ombre de la guillotine,1913
1,2844,Drama,Fantômas - À l'ombre de la guillotine,1913
2,4936,Comedy,The Bank,1915
3,4936,Short,The Bank,1915
4,4972,Drama,The Birth of a Nation,1915


In [8]:
# Import USERS.dat 

header_users= ["userID", "twitter_id"]
user_df=pd.read_csv('users.dat',sep='::',engine='python',names=header_users)
user_df.head()

,userID,twitter_id
0,1,18405182
1,2,31260677
2,3,138805259
3,4,292919324
4,5,575170669


In [9]:

#IMPORT RATING.dat
header_rating = ["userID" , "ID", "rating" ,"timestamp"]
rating_df =pd.read_csv('ratings.dat',sep='::',engine='python',names=header_rating)
rating_df.head()

,userID,ID,rating,timestamp
0,1,1074638,7,1365029107
1,1,1853728,8,1366576639
2,2,104257,8,1364690142
3,2,1259521,8,1364118447
4,2,1991245,7,1364117717


# MERGE DATA SETS into single dataframe

In [10]:
#MERGE
df_merge_movies_rating = pd.merge(movies_df_new, rating_df, on='ID')
df_merge_movies_rating.head()

,ID,GENRE,NAME,YEAR,userID,rating,timestamp
0,2844,Crime,Fantômas - À l'ombre de la guillotine,1913,11746,6,1362810972
1,2844,Drama,Fantômas - À l'ombre de la guillotine,1913,11746,6,1362810972
2,4936,Comedy,The Bank,1915,7705,8,1367791953
3,4936,Comedy,The Bank,1915,10136,5,1373070069
4,4936,Short,The Bank,1915,7705,8,1367791953


In [11]:
df_merge_all = pd.merge(df_merge_movies_rating,user_df,on='userID')
#df_merge_all.to_csv('final.csv',header=True)
df_merge_all.head(10)

,ID,GENRE,NAME,YEAR,userID,rating,timestamp,twitter_id
0,2844,Crime,Fantômas - À l'ombre de la guillotine,1913,11746,6,1362810972,87087710
1,2844,Drama,Fantômas - À l'ombre de la guillotine,1913,11746,6,1362810972,87087710
2,11717,Action,The Son of Tarzan,1920,11746,6,1365916918,87087710
3,11717,Adventure,The Son of Tarzan,1920,11746,6,1365916918,87087710
4,14538,Comedy,Three Ages,1923,11746,9,1363153767,87087710
5,15163,Action,The Navigator,1924,11746,9,1362807302,87087710
6,15163,Comedy,The Navigator,1924,11746,9,1362807302,87087710
7,15324,Comedy,Sherlock Jr.,1924,11746,9,1362422058,87087710
8,15324,Fantasy,Sherlock Jr.,1924,11746,9,1362422058,87087710
9,15400,Adventure,The Thief of Bagdad,1924,11746,8,1362209005,87087710


# Average of all ratings is used 
# Pandas SQL is used to generate the dataframe 

In [13]:
q1 = """SELECT YEAR , GENRE , AVG(RATING) avg_rating 
        FROM df_merge_all
        group by YEAR , GENRE
        order by YEAR DESC , avg_rating desc 
         """
df_movies_sql = ps.sqldf(q1, locals())
df_movies_sql.head()

,YEAR,GENRE,avg_rating
0,2013,Short,9.054795
1,2013,History,8.303030
2,2013,Sport,8.180212
3,2013,Documentary,7.855263
4,2013,Family,7.656269


In [14]:
#df_movies_sql1=df_movies_sql.groupby('YEAR')
df_final = df_movies_sql.sort_values('avg_rating', ascending=False).drop_duplicates('YEAR').sort_index()
df_final.head(10)

,YEAR,GENRE,avg_rating
0,2013,Short,9.054795
23,2012,Western,8.465578
46,2011,Short,8.578947
69,2010,Musical,7.948718
92,2009,Western,10.000000
114,2008,Music,8.333333
136,2007,Short,9.500000
158,2006,Music,8.031250
180,2005,Music,8.205128
202,2004,Biography,8.267176


In [16]:
# Year and most populat GENRE 
df_final[['YEAR','GENRE']].head(10)

,YEAR,GENRE
0,2013,Short
23,2012,Western
46,2011,Short
69,2010,Musical
92,2009,Western
114,2008,Music
136,2007,Short
158,2006,Music
180,2005,Music
202,2004,Biography
